# CberEmissionTrack guide
First of all, you need to install the package

As this project is intensively updating, in order to utilize SberEmisisonTrack correctly it is recommended firstly to delete the old version and then to start installation of the new one


In [1]:
!pip uninstall SberEmissionTrack -y
!pip install git+git://github.com/vladimir-laz/AIRIEmisisonTracker.git

Found existing installation: SberEmissionTrack 1.0.0
Uninstalling SberEmissionTrack-1.0.0:
  Successfully uninstalled SberEmissionTrack-1.0.0
  Cloning git://github.com/vladimir-laz/AIRIEmisisonTracker.git to /tmp/pip-req-build-j2vw3n9r
  Running command git clone -q git://github.com/vladimir-laz/AIRIEmisisonTracker.git /tmp/pip-req-build-j2vw3n9r
  Created wheel for SberEmissionTrack: filename=SberEmissionTrack-1.0.0-py3-none-any.whl size=26078 sha256=9732a1906ba72dc6a5333865577eb8c6f29a00c32e1093cff936fb940907baa6
  Stored in directory: /tmp/pip-ephem-wheel-cache-2yc49c_2/wheels/67/e7/41/7ae3b1d2339881a7b0688acdc1bbcca48d205b78766f2ad749
Successfully built SberEmissionTrack


In [2]:
import SberEmissionTrack
import pandas as pd

If such an error appears: 

```python
ContextualVersionConflict: (tzlocal 1.5.1 (/usr/local/lib/python3.7/dist-packages), Requirement.parse('tzlocal!=3.*,>=2.0'), {'APScheduler'})
```

Then just restart the kernel

You can observe all available cpu & gpu devices by running command below:

If there are no any available gpus then gpu power consumption will be set to zero

If there are no any available cpus then cpu power consumption will be set to base cpu consumption which is equal to 100Watt

In [3]:
SberEmissionTrack.available_devices()

Seeable cpu device(s):
        Intel(R) Xeon(R) CPU @ 2.20GHz: 2 device(s)
Seeable gpu device(s):
        Tesla P100-PCIE-16GB: 1 device(s)


In [4]:
tracker = SberEmissionTrack.Tracker(project_name="Tracker guide",
                                    experiment_description="testing SberEmissionTrack",
                                    save_file_name="emission.csv",
                                    measure_period=3, # period to measure current power consumption
                                    emission_level=511.7942) #CO2 emission per every MWh in Russia
tracker.start()

# you code

tracker.stop()

/usr/local/lib/python3.7/dist-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/usr/local/lib/python3.7/dist-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


The CO2 emission results are available at "emission.csv" file

In [5]:
pd.read_csv('emission.csv')

,project_name,experiment_description,time(s),power_consumption(kWTh),CO2_emissions(kg)
0,Tracker guide,testing SberEmissionTrack,0.573439,0.000006,0.000003


Also, all the fuctions and classes have detailed description:

In [6]:
help(SberEmissionTrack.Tracker)

Help on class Tracker in module SberEmissionTrack.emission_track:

class Tracker(builtins.object)
 |  Tracker(project_name='Deafult project name', experiment_description='no experiment description', save_file_name='emission.csv', measure_period=2, emission_level=511.7942)
 |  
 |  In order to calculate gpu & cpu power consumption correctly you should create the 'Tracker' before any gpu or cpu usage
 |  For every new calculation create a new “Tracker.”
 |  
 |  ----------------------------------------------------------------------
 |  Use example:
 |  
 |  import SberEmissionTrack.Tracker
 |  tracker = SberEmissionTrack.Tracker()
 |  
 |  tracker.start()
 |  
 |  *your gpu calculations*
 |  
 |  tracker.stop()
 |  ----------------------------------------------------------------------
 |  
 |  Methods defined here:
 |  
 |  __init__(self, project_name='Deafult project name', experiment_description='no experiment description', save_file_name='emission.csv', measure_period=2, emission_leve